# 서울시 교통사고 시각화

In [1]:
import pandas as pd

In [4]:
traffic_Seoul = pd.read_csv('../data/newSeoul_2005_2019.csv')
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [6]:
traffic_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      4500 non-null   int64 
 1   월       4500 non-null   int64 
 2   자치구명    4500 non-null   object
 3   발생건수    4500 non-null   int64 
 4   사망자수    4500 non-null   int64 
 5   부상자수    4500 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 211.1+ KB


In [5]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul.년도==2019]
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [9]:
traffic_Seoul_2019.reset_index(drop=True, inplace=True)

In [10]:
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158


In [20]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
traffic_analysis = traffic_Seoul_2019.pivot_table(
    ['발생건수','부상자수','사망자수'], 
    aggfunc=sum,
    index='자치구명'
    )
# pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수','부상자수','사망자수']], aggfunc=sum, index='자치구명')
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [21]:
# 위도, 경도 데이터 불러오기
seoul_limit = pd.read_csv('../data/seoul.csv', encoding='euc-kr')
seoul_limit.head()

,area,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [22]:
seoul_limit.columns=['자치구명','위도','경도']
seoul_limit.columns

Index(['자치구명', '위도', '경도'], dtype='object')

In [ ]:
seoul_limit.set_index()

In [26]:
# traffic_analysis와 seoul_limit 병합하기
data_result = pd.merge(traffic_analysis, seoul_limit, on='자치구명')

In [27]:
data_result.head()

,자치구명,발생건수,부상자수,사망자수,위도,경도
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [28]:
data_result.tail()

,자치구명,발생건수,부상자수,사망자수,위도,경도
20,용산구,1187,1614,10,126.9904,37.53254
21,은평구,1119,1464,7,126.9292,37.60278
22,종로구,1133,1559,6,126.9793,37.57323
23,중구,1184,1615,5,127.0032,37.55273
24,중랑구,1620,2122,12,127.0928,37.60652


In [29]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   위도      25 non-null     float64
 5   경도      25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


> int type은 읽을 수 없으니 float으로 바꿔주자

In [31]:
# folium을 사용하기 위해 int > float 변환하기
data_result[['발생건수','부상자수','사망자수']] = data_result[['발생건수','부상자수','사망자수']].astype('float')

In [32]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   위도      25 non-null     float64
 5   경도      25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구의 구청을 표시하기

In [44]:
import folium


map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

for name, lat, lng in zip(data_result.자치구명+'청', data_result.경도, data_result.위도):
    # print(name, lat, lng)
    folium.Marker([lat,lng], popup=name).add_to(map)

# for n in data_result.index:
#     folium.Marker(
#         [data_result.경도[n], data_result.위도[n]],
#         popup=data_result.자치구명[n]+'청',
#         ).add_to(map)

map

In [49]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   위도      25 non-null     float64
 5   경도      25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.9+ KB


In [88]:
# 자치구별 교통사고 발생건수를 표시하기
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

for name, lat, lng in zip(data_result.자치구명+'청', data_result.경도, data_result.위도):
    # print(name, lat, lng)
    folium.Marker([lat,lng], popup=name).add_to(map)

for n in data_result.index:
    folium.CircleMarker(
        [data_result.경도[n], data_result.위도[n]],
        radius=data_result['발생건수'][n] / 100,
        fill_color='grey',
        fill_opacity= (data_result['발생건수'][n] / data_result['발생건수'].max()) * 0.7,
        ).add_to(map)

map

In [107]:
# 자치구별 교통사고 사망건수를 표시하기
# 자치구별 교통사고 발생건수를 표시하기
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

for name, lat, lng in zip(data_result.자치구명+'청', data_result.경도, data_result.위도):
    # print(name, lat, lng)
    folium.Marker([lat,lng], popup=name).add_to(map)

# for n in data_result.index:
#     folium.CircleMarker(
#         [data_result.경도[n], data_result.위도[n]],
#         radius=data_result['사망자수'][n] * 2,
#         color='red',
#         fill_color='darkred',
#         fill_opacity= (data_result['사망자수'][n] / data_result['사망자수'].max()) * 0.7,
#         ).add_to(map)

folium.CircleMarker(
        [data_result.경도[n], data_result.위도[n]],
        radius=data_result['사망자수'][n] * 2,
        color='red',
        fill_color='darkred',
        fill_opacity= (data_result['사망자수'][n] / data_result['사망자수'].max()) * 0.7,
        ).add_to(map)
map

In [122]:
data_result[(data_result.사망자수 == data_result.사망자수.min()) | (data_result.사망자수 == data_result.사망자수.max())]

,자치구명,발생건수,부상자수,사망자수,위도,경도
3,강서구,1829.0,2491.0,20.0,126.8496,37.55094
5,광진구,973.0,1316.0,3.0,127.0824,37.53862


> 최소, 최대값의 인덱스만 뽑아와서 활용하자

In [125]:
# 자치구별 교통사고 사망건수를 표시하기
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

for name, lat, lng in zip(data_result.자치구명+'청', data_result.경도, data_result.위도):
    # print(name, lat, lng)
    folium.Marker([lat,lng], popup=name).add_to(map)

# 최대 3, 최소 5
folium.CircleMarker(
    [data_result.경도[3], data_result.위도[3]],
    radius=data_result['사망자수'][3] * 3,
    color='red',
    fill_color='darkred',
    fill_opacity= (data_result['사망자수'][3] / data_result['사망자수'].max()) * 0.7,
).add_to(map)
folium.CircleMarker(
    [data_result.경도[5], data_result.위도[5]],
    radius=data_result['사망자수'][5] * 3,
    color='red',
    fill_color='darkred',
    fill_opacity= (data_result['사망자수'][5] / data_result['사망자수'].max()) * 0.7,
).add_to(map)

map